In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
from tqdm import tqdm
import lightgbm as lgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re

from statsmodels.formula.api import ols
import statsmodels.api as sm
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
from statsmodels.api import qqplot 
from scipy.stats import shapiro,ttest_ind
from statsmodels.stats.diagnostic import het_breuschpagan,linear_rainbow  
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [2]:
data = pd.read_csv('All_data.csv')

grades = pd.read_csv('All_grades.csv')

In [3]:
users = grades.user_id.tolist()

In [4]:
user_id = []
for i in data['Description']:
    try:
        discovered_id = i.split("'")[1].replace('.',"")
        _ = int(discovered_id)
        
        user_id.append(discovered_id)
        
    except:
        user_id.append('error')
 

In [5]:
data['user_id'] = user_id
data_users = data.user_id.unique().tolist()

In [6]:
data_users_int = []
for i in data_users:
    try: 
        data_users_int.append(int(i))
    except: 
        continue

In [7]:
not_students = list(set(data_users_int) - set(users))

In [8]:
not_students.append('error')
not_students = [str(i) for i in not_students]
not_students[:10]

['0', '5120', '2', '5123', '5125', '5130', '7179', '5133', '5135', '2064']

In [9]:
data=data[~data.user_id.isin(not_students)]

In [10]:
data = data.reset_index(drop = True)

In [11]:
data['Time'] = pd.to_datetime(data['Time'], dayfirst=True)

In [12]:
def extract_date(datetime):
    return datetime.day, datetime.week, datetime.month,datetime.year

data['day'], data['week'], data['month'], data['year'] = zip(*data['Time'].map(extract_date))

data['week'] = data['week'] - (data['week'].min() -1)

In [13]:
interval_def_template = {
       'start_date': '',
       'end_date': '',
       'student_ids' : {}
}

In [54]:
def filter_intervals_data(data, time='week'):

    user_activity_dict = {}

    interval_range = sorted(data[time].unique())
    unique_event_dict = {item : 0 for item in data['Event name'].unique()}

    for interval in tqdm(interval_range):
        interval_def_template = {
           'start_date': '',
           'end_date': '',
           'student_ids' : {}
        }

        dict_template = {}

        filtered_data = data[data[time]==interval]
        int_start, int_end = str(filtered_data.Time.min().date()), str(filtered_data.Time.max().date())

        interval_def_template['start_date'], interval_def_template['end_date'] = int_start, int_end

        for user_id in filtered_data.user_id.unique():
            
            user_filtered_events = filtered_data[filtered_data.user_id==user_id]['Event name'].value_counts()
            dict_template[user_id] = dict(unique_event_dict)

            for event, count in dict(user_filtered_events).items():
                dict_template[user_id][event] = count
            
            dict_template[user_id]['type'] = filtered_data[filtered_data.user_id==user_id]['type'].unique()[0]
            dict_template[user_id]['level'] = filtered_data[filtered_data.user_id==user_id]['level'].unique()[0]

        interval_def_template['student_ids'] = dict_template

        user_activity_dict[f"{time}_{interval}"] = interval_def_template
        
    return user_activity_dict
    


In [55]:
user_filtered_events = filter_intervals_data(data, time='week')

100%|██████████| 18/18 [00:34<00:00,  1.91s/it]


In [56]:
total_user_activity = []
time_interval_list = []

for time_interval, user_events in user_filtered_events.items():
    total_user_activity.append(
        pd.DataFrame(user_filtered_events[f'{time_interval}']['student_ids'].values(), index=user_filtered_events[f'{time_interval}']['student_ids'].keys()))
    
for time_interval, user_events in user_filtered_events.items():
    time_list = [time_interval for i in range(len(user_filtered_events[time_interval]['student_ids'].keys()))]
    time_interval_list+=time_list

In [57]:
k = pd.concat(total_user_activity).reset_index()

In [58]:
time_interval_df = pd.DataFrame(time_interval_list, columns = ['time_interval'])

In [59]:
final = pd.concat([time_interval_df, k], axis=1)

In [60]:
final.rename(columns={'index': 'user_id'},inplace =True)

In [61]:
final.user_id = final.user_id.astype('int')

In [62]:
final = final.merge(grades, on  = 'user_id',how = 'left')

In [63]:
final.dropna(inplace = True)

In [64]:
final.head()

,time_interval,user_id,The status of the submission has been viewed.,Course module viewed,Feedback viewed,Course viewed,Grade user report viewed,User profile viewed,Course activity completion updated,Course user report viewed,...,Subscription created,Subscription deleted,Group member added,Choice answer deleted,Grade item updated,Recent activity viewed,An online text has been uploaded.,type,level,Grades
0,week_1,1943,0,16,0,46,0,22,15,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0
1,week_1,2666,0,8,0,14,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,3.7
2,week_1,1931,0,1,0,4,0,0,0,1,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0
3,week_1,2013,0,46,0,49,1,2,2,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,3.7
4,week_1,2385,0,13,0,23,0,0,2,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0


In [223]:
test = final.copy()

In [224]:
# for i in test.columns:
#     if sum(test[i].values == 0) > 0.5*len(test):
#         test.drop(i,axis=1,inplace = True)

In [225]:
test = test.drop('user_id', 1).reset_index(drop = True)

In [226]:
test

,time_interval,The status of the submission has been viewed.,Course module viewed,Feedback viewed,Course viewed,Grade user report viewed,User profile viewed,Course activity completion updated,Course user report viewed,Zip archive of folder downloaded,...,Subscription created,Subscription deleted,Group member added,Choice answer deleted,Grade item updated,Recent activity viewed,An online text has been uploaded.,type,level,Grades
0,week_1,0,16,0,46,0,22,15,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0
1,week_1,0,8,0,14,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,3.7
2,week_1,0,1,0,4,0,0,0,1,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0
3,week_1,0,46,0,49,1,2,2,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,3.7
4,week_1,0,13,0,23,0,0,2,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7647,week_18,1,1,1,2,2,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,3.7
7648,week_18,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,4.0
7649,week_18,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,3.3
7650,week_18,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,4.0


In [227]:
final.time_interval.value_counts()

week_5     453
week_4     451
week_2     451
week_3     450
week_13    450
week_15    449
week_17    449
week_10    448
week_14    444
week_7     443
week_9     441
week_16    441
week_1     438
week_11    437
week_8     435
week_6     423
week_12    419
week_18    130
Name: time_interval, dtype: int64

In [228]:
# def determine_grade(score):

#     if score  <= 4 and score >= 3.7:
#         return 'A'
#     elif score < 3.7 and score >= 2.7 :
#         return 'B'
#     elif score < 2.7 and score >= 2:
#         return "C"
#     elif score < 2 and score >= 1.3:
#         return "D"
#     else:
#         return 'F'

In [229]:
test.Grades = np.where(test.Grades < 3, 'Low Performance', 'High Performance')

In [230]:
test.Grades.value_counts()

High Performance    5092
Low Performance     2560
Name: Grades, dtype: int64

In [231]:
test

,time_interval,The status of the submission has been viewed.,Course module viewed,Feedback viewed,Course viewed,Grade user report viewed,User profile viewed,Course activity completion updated,Course user report viewed,Zip archive of folder downloaded,...,Subscription created,Subscription deleted,Group member added,Choice answer deleted,Grade item updated,Recent activity viewed,An online text has been uploaded.,type,level,Grades
0,week_1,0,16,0,46,0,22,15,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
1,week_1,0,8,0,14,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
2,week_1,0,1,0,4,0,0,0,1,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
3,week_1,0,46,0,49,1,2,2,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
4,week_1,0,13,0,23,0,0,2,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7647,week_18,1,1,1,2,2,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,High Performance
7648,week_18,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,High Performance
7649,week_18,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,High Performance
7650,week_18,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,grad,High Performance


In [232]:
for i in test.time_interval.unique():
    print(i)

week_1
week_2
week_3
week_4
week_5
week_6
week_7
week_8
week_9
week_10
week_11
week_12
week_13
week_14
week_15
week_16
week_17
week_18


In [233]:
intervals = []
for i in test.time_interval.unique()[:-3]:
    intervals.append(test[test.time_interval == i].index[-1])

In [235]:
intervals

[437,
 888,
 1338,
 1789,
 2242,
 2665,
 3108,
 3543,
 3984,
 4432,
 4869,
 5288,
 5738,
 6182,
 6631]

In [236]:
test = test.drop('time_interval',1 )

In [237]:
test.head()

,The status of the submission has been viewed.,Course module viewed,Feedback viewed,Course viewed,Grade user report viewed,User profile viewed,Course activity completion updated,Course user report viewed,Zip archive of folder downloaded,A submission has been submitted.,...,Subscription created,Subscription deleted,Group member added,Choice answer deleted,Grade item updated,Recent activity viewed,An online text has been uploaded.,type,level,Grades
0,0,16,0,46,0,22,15,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
1,0,8,0,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
2,0,1,0,4,0,0,0,1,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
3,0,46,0,49,1,2,2,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance
4,0,13,0,23,0,0,2,0,0,0,...,0,0,0,0,0,0,0,Quantitative,undergrad,High Performance


In [238]:
for i in intervals:
    print(test.loc[:i].shape)

(438, 63)
(889, 63)
(1339, 63)
(1790, 63)
(2243, 63)
(2666, 63)
(3109, 63)
(3544, 63)
(3985, 63)
(4433, 63)
(4870, 63)
(5289, 63)
(5739, 63)
(6183, 63)
(6632, 63)


In [239]:
X = pd.get_dummies(test.drop('Grades', axis = 1))
Y = test.Grades

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2,random_state = 42)

In [241]:
intervals[6:]

[3108, 3543, 3984, 4432, 4869, 5288, 5738, 6182, 6631]

In [245]:
for i in intervals[6:]:
    X = pd.get_dummies(test.loc[:i].drop('Grades', axis = 1))
    Y = test.loc[:i].Grades
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2,random_state = 42)
    logit = LogisticRegression(max_iter=1e6).fit(X_train,y_train)
    Y0_logit = logit.predict_proba(X_train)[:,1]
    Y1_logit = logit.predict_proba(X_test)[:,1]
    print("ROC AUC Logistic Regression Train", roc_auc_score(y_train,Y0_logit))
    print("ROC AUC Logistic Regression Test", roc_auc_score(y_test,Y1_logit), '\n')x

ROC AUC Logistic Regression Train 0.7252541241331784
ROC AUC Logistic Regression Test 0.7157045095630812 

ROC AUC Logistic Regression Train 0.7331861592867882
ROC AUC Logistic Regression Test 0.713193899782135 

ROC AUC Logistic Regression Train 0.735441342543899
ROC AUC Logistic Regression Test 0.7141881870226363 

ROC AUC Logistic Regression Train 0.7387101388519083
ROC AUC Logistic Regression Test 0.7207614917805234 

ROC AUC Logistic Regression Train 0.7465101508810756
ROC AUC Logistic Regression Test 0.7052864096545843 

ROC AUC Logistic Regression Train 0.7389307815407424
ROC AUC Logistic Regression Test 0.7167544437129691 

ROC AUC Logistic Regression Train 0.7383439409771698
ROC AUC Logistic Regression Test 0.7293186971758401 

ROC AUC Logistic Regression Train 0.7420329603871127
ROC AUC Logistic Regression Test 0.7193783434790629 

ROC AUC Logistic Regression Train 0.7384925956944569
ROC AUC Logistic Regression Test 0.7246876979602179 

